In [ ]:
'checkpoints_student/T=10, alpha=0.5, dropout_hidden=0.0, dropout_input=0.0, lr=0.01, lr_decay=0.95, momentum=0.9, weight_decay=0.001_pruning_0.1_final.tar'

In [2]:
import numpy as np
import math

%matplotlib inline
import matplotlib.pyplot as plt

import pickle
import argparse
import time
import itertools
from copy import deepcopy
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import csv
import sys
#sys.path.append('/content/KD')
# Import the module
import networks
import utils

%load_ext autoreload
%autoreload 2

In [3]:
use_gpu = True    # set use_gpu to True if system has gpu
gpu_id = 0        # id of gpu to be used
cpu_device = torch.device('cpu')
# fast_device is where computation (training, inference) happens
fast_device = torch.device('cpu')
if use_gpu:
    os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2'    # set visible devices depending on system configuration
    fast_device = torch.device('cuda:' + str(gpu_id))

In [6]:
def reproducibilitySeed():
    """
    Ensure reproducibility of results; Seeds to 0
    """
    torch_init_seed = 0
    torch.manual_seed(torch_init_seed)
    numpy_init_seed = 0
    np.random.seed(numpy_init_seed)
    if use_gpu:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

reproducibilitySeed()

In [4]:
import torchvision
import torchvision.transforms as transforms

# Set up transformations for CIFAR-10
transform_train = transforms.Compose(
    [
        transforms.RandomCrop(32, padding=4),  # Augment training data by padding 4 and random cropping
        transforms.RandomHorizontalFlip(),     # Randomly flip images horizontally
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))  # Normalization for CIFAR-10
    ]
)

transform_test = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))  # Normalization for CIFAR-10
    ]
)

# Load CIFAR-10 dataset
train_val_dataset = torchvision.datasets.CIFAR10(root='./CIFAR10_dataset/', train=True,
                                            download=True, transform=transform_train)

test_dataset = torchvision.datasets.CIFAR10(root='./CIFAR10_dataset/', train=False,
                                            download=True, transform=transform_test)

# Split the training dataset into training and validation
num_train = int(0.95 * len(train_val_dataset))  # 95% of the dataset for training
num_val = len(train_val_dataset) - num_train  # Remaining 5% for validation
train_dataset, val_dataset = torch.utils.data.random_split(train_val_dataset, [num_train, num_val])

# DataLoader setup
batch_size = 128
train_val_loader = torch.utils.data.DataLoader(train_val_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [16]:
# Path to the saved model
teacher_path = "checkpoints_teacher/dropout_hidden=0.0, dropout_input=0.0, lr=0.01, lr_decay=0.95, momentum=0.9, weight_decay=0.001_final.tar"

# Initialize the network
teacher_net = networks.TeacherNetwork()
teacher_net = teacher_net.to(fast_device)

# Load the checkpoint
checkpoint = torch.load(teacher_path)

# Load the state dictionary into the model
teacher_net.load_state_dict(checkpoint['model_state_dict'])

# pre-trained teacher accuracy
reproducibilitySeed()
_, test_accuracy = utils.getLossAccuracyOnDataset(teacher_net, test_loader, fast_device)
print('test accuracy: ', test_accuracy)

c:\Users\daniel\anaconda3\envs\Classifier\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\daniel\anaconda3\envs\Classifier\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\daniel\AppData\Local\Temp\ipykernel_49368\4278446609.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.c

test accuracy:  0.8616


In [ ]:
from quantize_neural_net import QuantizeNeuralNet
pruning_factors = [i/20 for i in range(1, 11)]


# Path to the saved model
student_path = "checkpoints_student/T=10, alpha=0.5, dropout_hidden=0.0, dropout_input=0.0, lr=0.01, lr_decay=0.95, momentum=0.9, weight_decay=0.001_pruning_0.4_final.tar"

# Initialize the network
student_model = networks.StudentNetwork(.4, teacher_net)
student_model = student_model.to(fast_device)

# Load the checkpoint
checkpoint = torch.load(student_path)

student_model.load_state_dict(checkpoint['model_state_dict'])

# Ensure reproducibility and evaluate pre-trained teacher accuracy
reproducibilitySeed()


_, test_accuracy = utils.getLossAccuracyOnDataset(student_model, test_loader, fast_device)
print('Test accuracy:', test_accuracy)

quantizer = QuantizeNeuralNet(student_model.model.model,
    'resnet18',  # Default from `-model`
    batch_size=128,  # Default from `--batch_size`
    data_loader=train_loader,
    mlp_bits=4,  # Default from `--bits`
    cnn_bits=4,  # Default from `--bits`
    ignore_layers=[],  # Default from `--ignore_layer`
    mlp_alphabet_scalar=1.16,  # Default from `--scalar`
    cnn_alphabet_scalar=1.16,  # Default from `--scalar`
    mlp_percentile=1,  # Default from `--percentile`
    cnn_percentile=1,  # Default from `--percentile`
    reg=None,  # Default from `--regularizer`
    lamb=0.1,  # Default from `--lamb`
    retain_rate=0.25,  # Default from `--retain_rate`
    stochastic_quantization=False,  # Default from `--stochastic_quantization`
    device=fast_device
)

quantized_model = quantizer.quantize_network()

_, test_accuracy = utils.getLossAccuracyOnDataset(quantized_model, test_loader, fast_device)
print('Test accuracy:', test_accuracy)

c:\Users\daniel\anaconda3\envs\Classifier\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\daniel\anaconda3\envs\Classifier\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\daniel\AppData\Local\Temp\ipykernel_49368\1596229990.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.c

Test accuracy: 0.8604


In [22]:
def count_parameters(model):
    """
    Counts the total number of trainable parameters in a PyTorch model.

    Args:
        model (torch.nn.Module): The model whose parameters need to be counted.

    Returns:
        int: Total number of trainable parameters.
    """
    return sum((p.data != 0).sum().item() for p in model.parameters() if p.requires_grad)


def count_zero_parameters(model):
    """
    Counts the number of trainable parameters that are exactly zero in a PyTorch model.

    Args:
        model (torch.nn.Module): The model whose zero parameters need to be counted.

    Returns:
        int: Total number of trainable parameters that are exactly zero.
    """
    return sum((p.data == 0).sum().item() for p in model.parameters() if p.requires_grad)

In [23]:
count_parameters(student_model)

6712822

In [42]:
from quantize_neural_net import QuantizeNeuralNet


In [48]:
quantized_model = quantizer.quantize_network()

Layer indices to quantize [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Total number of layers to quantize 21

Quantizing layer with index: 0
Quantization progress: 0 out of 20

shape of W: torch.Size([64, 3, 7, 7])
shape of analog_layer_input: torch.Size([896, 147])
shape of quantized_layer_input: torch.Size([896, 147])
The number of groups: 1



100%|██████████| 147/147 [00:00<00:00, 549.03it/s]


The quantization error of layer 0 is 25.63368797302246.
The relative quantization error of layer 0 is 0.07626139372587204.


Quantizing layer with index: 1
Quantization progress: 1 out of 20

shape of W: torch.Size([64, 64, 3, 3])
shape of analog_layer_input: torch.Size([384, 576])
shape of quantized_layer_input: torch.Size([384, 576])
The number of groups: 1



100%|██████████| 576/576 [00:00<00:00, 2098.73it/s]


The quantization error of layer 1 is 3.9425508975982666.
The relative quantization error of layer 1 is 0.05762189254164696.


Quantizing layer with index: 2
Quantization progress: 2 out of 20

shape of W: torch.Size([64, 64, 3, 3])
shape of analog_layer_input: torch.Size([384, 576])
shape of quantized_layer_input: torch.Size([384, 576])
The number of groups: 1



...

The quantization error of layer 18 is 1.88277268409729.
The relative quantization error of layer 18 is 0.1320144683122635.


Quantizing layer with index: 19
Quantization progress: 19 out of 20

shape of W: torch.Size([512, 512, 3, 3])
shape of analog_layer_input: torch.Size([128, 4608])
shape of quantized_layer_input: torch.Size([128, 4608])
The number of groups: 1



100%|██████████| 4608/4608 [00:01<00:00, 3428.54it/s]


The quantization error of layer 19 is 0.6852036714553833.
The relative quantization error of layer 19 is 0.14334264397621155.


Quantizing layer with index: 20
Quantization progress: 20 out of 20

The number of groups: 1



100%|██████████| 512/512 [00:00<00:00, 3313.58it/s]


The quantization error of layer 20 is 20.633893966674805.
The relative quantization error of layer 20 is 0.19272233545780182.

